## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"
[image2]: ./test_images/test1.jpg "Road Transformed"
[image3]: ./examples/binary_combo_example.jpg "Binary Example"
[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[video1]: ./project_video.mp4 "Video"

### Here I will consider the [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points individually and describe how I addressed each point in my implementation.  

There are two set of code. The first one is consist of different `.py` files whose names are `camera_calibration.py`, `image_gen.py`, `tracker.py`, `video_process.py`. Another is a `.ipynb` file which is easy to show the result of each section. It contains code of `camera_calibration.py`, `tracker.py`,  revised `image_gen.py` for single image process. The result of each stage of test images are also saved in `output_images` folder. The result video is in this repository. Following are explanation of those files.

* `camera_calibration.py` is used for calibrating camera and save result of `cv2.calibrateCamera()` to a pickle file
* `image_gen.py` is used for process all test image in the `test_image` folder. 
* `tracker.py` save some important parameters and has a function outputing an array of x coordinate of left and right center points for each level.
* `video_process.py` is used for process video

### 1. Camera Calibration

The code for this step is contained in the file called `camera_calibration.py`. 

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function . The result is shown in the cell [3] in `Final_find_lane.ipynb` file.
![alt text](./analysis/undistort.png)


### 2. Pipeline (single images)

#### a. Provide an example of a distortion-corrected image.

To make a better tracking the lane line in the image, I create a class called 'tracker' (in cell [4] in the jupyter notebook)to save some important parameters, like window width and height of convoluiton for finding lane, searching margin, and also the ratio that converts pixel to real distance in meter. Class tracker also has a function whose output is an array containing the pair of x coordinate of left and right line for each level. As mentioned above, undistorted result is shown in in the cell [3] in `Final_find_lane.ipynb` file.



#### b.  Create a thresholded binary image.  

I defined different functions in cell [5] for absolute sobel threshold binary, magnitude threshold binary, dirction threshold binary and combined color (HSV and HLS color space) threshold binary. After many times of experimenting, I used a combination of color and sobel threshold of x and y to generate a binary image.  Each function result and final binary result are all shown in cell [6]. Final result code is in the line 29 to 33 in cell [6].

Gradient threshold binary image
![alt text](./analysis/grad_thresh.png) 

Color threshold binary image
![alt text](./analysis/color_thresh.png)
#### c.  Perspective transform

The code for my perspective transform is in cell [7], which appears in lines 1 through 8 in the file `example.py` (output_images/examples/example.py) (or, for example, in the 3rd code cell of the IPython notebook).  The `warper()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
bot_width = 0.8
top_width = 0.091
top_height = .62
bot_height = .935
offset = img_size[0]*.25

src = np.float32([[img_size[0]*(0.5-top_width/2), img_size[1]*top_height],
				  [img_size[0]*(0.5+top_width/2), img_size[1]*top_height],
				  [img_size[0]*(0.5+bot_width/2), img_size[1]*bot_height],
				  [img_size[0]*(0.5-bot_width/2), img_size[1]*bot_height],])
                  
dst = np.float32([[offset,0],
            [img_size[0]-offset,0],
            [img_size[0]-offset,720],
            [offset,720]])
```
I selected source points by treating the selected area as a trapzoid and set them using ratio to the image shape. In order to not select too much unrelated part of image, I set offset to determine the destination points. With `src` and `dst`, I used cv2.getPerspectiveTransform() to calculate transformation matrix and inverse transformation matrix.

![alt text](./analysis/perspective_transform.png)

#### d. Identifying lane-line pixels and fit their positions with a polynomial?

To identify lane-line pixels, as mentioned before, I divide the image in to levels with a certain height (80 pixels) and create a function in tracker class to utilize `np.convolve` to x coordinate of center points of left and right lane line for each level. All these windows are shown in green in the cell [8].

![alt text](./analysis/tracking.png)

To draw a smoother line that covering detected lane line, I use `np.polifit` , left and right center points to get coefficients of lane curve which is treated as second order. Using these coefficient to find left and right line point for every y. Then draw left, right and area between them in a black background. As shown in  cell [9]. 

![alt text](./analysis/area_between_lines.png)

Since we already have inverse perspective transformation matrix, we convert the perspective of this detected area back to image perspective. With `cv2.addWeighted` function, I cover the detected area on to the original image. As show in cell [10].

![alt text](./analysis/.png)

#### e. Calculating the radius of curvature of the lane and the position of the vehicle with respect to center.

To find the curvature, I wrote code in cell [10] from line 4 through 7 and for vehicle position deviate from center, it is calculate from line 9 through 15.

#### f. Final result

The final result is shown in the cell [11]. 

![alt text](./analysis/final.png)

### Pipeline (video)


Here's a [link to my video result](https://github.com/DavidYRB/CarND-Advanced-Find-Lane). The video is named `output_video.mp4`

---

### Discussion

In current result, some image still cannot be tracked very accurately. As we can see from the output image, in some frames, we lost the lane line track. I think binary threshold is one of the key process that needs to be improved. With different gradient combination, different color space combination. could cause different result. 

Another point is the choice of source points and destination points for perspective transformation. This also leads to another question? Since cameras mounted on the car are different, position of lane lines in the image could be different and zoomed. This could cause no detection for some part of the line. 

Last one is the shade and different lumination condition could cause drastic difference on detection. I think this is a still computer vision problem.
